In [ ]:
import bokeh.plotting as bkp
import bokeh.models as bkm
bkp.output_notebook()

import os
os.environ['ARRAY_MODULE'] = 'numpy'
from gigablochs import bloch, rf, animation
from gigablochs import xp, asnumpy
import numpy as np

In [ ]:
duration = 0.20 # seconds
dt = 0.00025 # seconds

flip_angle = 180 # degrees
rf_duration = 0.040 # seconds
rf_bandwidth = np.linspace(250, 2000, 9) # Hz
B1_inhomogeneity = np.linspace(0.01, 1, 100) # fraction of B1

off_resonance = 1600 # Hz
spectrum_lines = 600

T1 = 1.5 # seconds
T2 = 0.1 # seconds

In [ ]:
time = np.arange(0, duration, dt) # seconds
rf_time = np.arange(-rf_duration / 2, rf_duration / 2, dt)

rf_pulse = np.array([rf.sinc_pulse(flip_angle, rf_duration, bw.item(), dt, phase_angle=-90) for bw in rf_bandwidth]).T
rf_sig = rf.extend(rf_pulse, duration, dt)
dfz = np.linspace(0, off_resonance, spectrum_lines) # Hz

B = bloch.construct_B_field(rf_sig, off_resonance=dfz, B1_sensitivity=B1_inhomogeneity)

In [ ]:
B.shape

In [ ]:
B.size * 4 / 1e9 # GB

In [ ]:
type(B)

In [ ]:
# plot RF with bokeh
plot = bkp.figure(width=800, height=400, title='RF pulse')
plot.line(asnumpy(rf_time * 1e3), asnumpy(rf_sig[:rf_time.shape[0], -1].real * 1e6), line_width=2)
plot.xaxis.axis_label = 'Time (ms)'
plot.yaxis.axis_label = 'RF Amplitude (µT)'
bkp.show(plot)

In [ ]:
mags = bloch.sim(B, T1, T2, duration, dt)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(asnumpy(time), asnumpy(mags[:, 1, 12, -1, 0]), line_width=2, legend_label='Mx', alpha=0.5)
plot.line(asnumpy(time), asnumpy(mags[:, 1, 12, -1, 1]), line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(asnumpy(time), asnumpy(mags[:, 1, 12, -1, 2]), line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization off-resonances with bokeh
title = 'Longitudinal Magnetization with Off-Resonance Pulse'
plot = bkp.figure(width=1000, height=500, title=title)
for offres in range(0, end := mags.shape[2], end // 10):
    alpha = 1 - offres / end
    plot.line(asnumpy(time), asnumpy(mags[:, 1, offres, -1, 2]), line_width=2,
              legend_label=f'{asnumpy(dfz[offres]):g} Hz',
              alpha=alpha, color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
plot.x_range = bkm.DataRange1d(start=0.01, end=0.05)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
band = 3
plot = bkp.figure(width=800, height=400, title=f'Flipped Magnetization Spectrum for {rf_bandwidth[band]} Hz bandwidth')
plot.line(asnumpy(dfz), asnumpy(mags.min(axis=0)[band, :, -1, -1]), line_width=2)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
bkp.show(plot)

In [ ]:
plot = bkp.figure(width=800, height=400, title='Flipped Magnetization Homogeneity')
plot.line(asnumpy(B1_inhomogeneity), asnumpy(mags.min(axis=0)[-1, 0, :, -1]), line_width=2)
plot.xaxis.axis_label = 'B1 Inhomogenrity'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
bkp.show(plot)

In [ ]:
freq = asnumpy(dfz)
B1 = asnumpy(B1_inhomogeneity)
minmag = asnumpy(mags.min(axis=0)[-1, ..., -1].T)

title = 'Inverted Magnetization Spectrum'
plot = bkp.figure(width=1000, height=500, title=title)
color_mapper = bkm.LinearColorMapper(palette='Viridis256', low=-1, high=1)
image = plot.image([minmag], y=[B1.min()], x=[freq.min()],
                   dh=[B1.max() - B1.min()],
                   dw=[freq.max() - freq.min()], color_mapper=color_mapper)
plot.xaxis.axis_label = 'Off-Resonance Frequency (Hz)'
plot.yaxis.axis_label = 'B1 Inhomogenrity'
plot.x_range = bkm.DataRange1d(start=freq.min(), end=freq.max())
plot.y_range = bkm.DataRange1d(start=B1.min(), end=B1.max())

# add colourbar
color_bar = bkm.ColorBar(color_mapper=color_mapper, location=(0, 0))
color_bar.title = 'Min Magnetization (ref M0)'
plot.add_layout(color_bar, 'right')

# bkp.output_file(f'{title}.html')
# bkp.save(plot)
bkp.show(plot)

In [ ]:
start = 0.01
end = 0.03
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(asnumpy(time[int(start // dt):int(end // dt)]), asnumpy(mags[int(start // dt):int(end // dt), 0, 0, -1, 0]), line_width=2, legend_label='Mx', alpha=0.5)
plot.line(asnumpy(time[int(start // dt):int(end // dt)]), asnumpy(mags[int(start // dt):int(end // dt), 0, 0, -1, 1]), line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(asnumpy(time[int(start // dt):int(end // dt)]), asnumpy(mags[int(start // dt):int(end // dt), 0, 0, -1, 2]), line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
# plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
asnumpy(time[int(start // dt):int(end // dt)]).shape

In [ ]:
anim_dt = 0.05
anim_duration = '~14'

anim_time, anim_mags = animation.downsample(asnumpy(mags[int(start // dt):int(end // dt), 0, 0, -1]), anim_dt, anim_duration)
anim_time, anim_beff = animation.downsample(asnumpy(B[int(start // dt):int(end // dt), 0, 0, -1]), anim_dt, anim_duration)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='Magnetization')
plot.line(anim_time, anim_mags[..., 0], line_width=2, legend_label='Mx', alpha=0.5)
plot.line(anim_time, anim_mags[..., 1], line_width=2, legend_label='My', color='orange', alpha=0.5)
plot.line(anim_time, anim_mags[..., 2], line_width=2, legend_label='Mz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'Magnetization (ref M0)'
# plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
# plot magnetization with bokeh
plot = bkp.figure(width=800, height=400, title='B effective')
plot.line(anim_time, anim_beff[..., 0], line_width=2, legend_label='Bx', alpha=0.5)
plot.line(anim_time, anim_beff[..., 1], line_width=2, legend_label='By', color='orange', alpha=0.5)
plot.line(anim_time, anim_beff[..., 2], line_width=2, legend_label='Bz', color='green')
plot.xaxis.axis_label = 'Time (s)'
plot.yaxis.axis_label = 'B effective (µT)'
# plot.x_range = bkm.DataRange1d(start=0, end=duration)
plot.legend.click_policy = 'hide'
bkp.show(plot)

In [ ]:
speed = animation.speed(asnumpy(time[int(start // dt):int(end // dt)]), anim_time)
speed

In [ ]:
animation.bloch_sphere(anim_mags, anim_beff, np.gradient(anim_time), speed,
                       traces=('magnetization',),
                       quality='high_quality')

In [ ]:
# save mags, B, dt, and dfz to compressed numpy file
# np.savez_compressed('mags.npz', mags=mags, B=B, dt=dt, dfz=dfz, B1=B1_inhomogeneity)